# Neural Networks: From Linear Regression to Deep Learning

<figure>
    <center> <img src="./images/neural_network_intro.png"  style="width:800px;height:200px;" ></center>
</figure>

## What You'll Learn

In the previous notebooks, we learned:
- **Model Representation**: $f_{w,b}(x) = wx + b$ (linear regression)
- **Cost Function**: How to measure prediction errors
- **Gradient Descent**: How to optimize parameters

But what if the relationship between input and output **isn't linear**?

This notebook introduces **Neural Networks** - the foundation of modern AI systems like ChatGPT, Claude, and image recognition.

### Learning Objectives
1. Understand why linear models fail on non-linear data
2. Build a simple neural network from scratch
3. Implement activation functions (sigmoid, ReLU)
4. Perform forward propagation
5. Understand backpropagation intuitively
6. Train a multi-layer network on real data

## 1. Setup and Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.style.use('./leonteq.mplstyle')
%matplotlib inline

# Set random seed for reproducibility
np.random.seed(42)

ImportError: cannot import name '_c_internal_utils' from partially initialized module 'matplotlib' (most likely due to a circular import) (C:\Python312\Lib\site-packages\matplotlib\__init__.py)

## 2. The Limitation of Linear Models

Let's create a dataset where the relationship between x and y is **non-linear**.

### Example: Housing Prices with Non-Linear Features

Imagine housing prices don't just increase linearly with size. Perhaps:
- Small houses (< 100m²) are cheap
- Medium houses (100-200m²) have a steep price increase
- Large houses (> 200m²) plateau in value

This is a **non-linear** relationship!

In [ ]:
# Generate non-linear data (sigmoid-like)
def generate_nonlinear_data(n_samples=100):
    """
    Generate housing data with non-linear relationship
    """
    x = np.linspace(0, 4, n_samples)
    # Non-linear function: sigmoid-like curve
    y = 400 / (1 + np.exp(-2*(x - 2))) + 100
    # Add some noise
    y += np.random.normal(0, 20, n_samples)
    return x, y

x_train, y_train = generate_nonlinear_data(100)

# Plot the data
plt.figure(figsize=(10, 6))
plt.scatter(x_train, y_train, alpha=0.5, label='Data Points')
plt.xlabel('Size (100 m²)')
plt.ylabel('Price (1000s CHF)')
plt.title('Non-Linear Housing Price Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Generated {len(x_train)} data points")
print(f"Size range: {x_train.min():.2f} to {x_train.max():.2f} (100 m²)")
print(f"Price range: {y_train.min():.0f} to {y_train.max():.0f} (1000s CHF)")

### Try Linear Regression (It Will Fail!)

Let's use our previous linear model $f_{w,b}(x) = wx + b$ and see what happens:

In [ ]:
# Simple linear regression using normal equation
def fit_linear_regression(x, y):
    """Fit linear model using least squares"""
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    
    w = np.sum((x - x_mean) * (y - y_mean)) / np.sum((x - x_mean)**2)
    b = y_mean - w * x_mean
    
    return w, b

w_linear, b_linear = fit_linear_regression(x_train, y_train)
y_pred_linear = w_linear * x_train + b_linear

# Calculate error
mse_linear = np.mean((y_train - y_pred_linear)**2)

# Plot results
plt.figure(figsize=(10, 6))
plt.scatter(x_train, y_train, alpha=0.5, label='Actual Data')
plt.plot(x_train, y_pred_linear, 'r-', linewidth=2, label=f'Linear Model (MSE={mse_linear:.0f})')
plt.xlabel('Size (100 m²)')
plt.ylabel('Price (1000s CHF)')
plt.title('Linear Regression on Non-Linear Data - Poor Fit!')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nLinear Model: f(x) = {w_linear:.2f}x + {b_linear:.2f}")
print(f"Mean Squared Error: {mse_linear:.2f}")
print(f"\n❌ The linear model cannot capture the non-linear relationship!")

## 3. Enter Neural Networks

### The Key Idea: Add Non-Linearity!

A neural network is just:
1. **Linear transformations** (like we did before)
2. **Non-linear activation functions** (the magic ingredient!)
3. **Stacked in layers** (to learn complex patterns)

### Architecture of a Simple Neural Network

```
Input → [Hidden Layer] → [Output Layer] → Prediction
  x   →  [w1, b1, σ]  →   [w2, b2, σ]  →     ŷ
```

Where:
- **w, b**: Weights and biases (like before)
- **σ**: Activation function (non-linear transformation)

### Common Activation Functions

In [ ]:
# Define activation functions
def sigmoid(z):
    """Sigmoid: squashes values to (0, 1)"""
    return 1 / (1 + np.exp(-z))

def relu(z):
    """ReLU: Rectified Linear Unit - max(0, z)"""
    return np.maximum(0, z)

def tanh(z):
    """Tanh: squashes values to (-1, 1)"""
    return np.tanh(z)

# Plot activation functions
z = np.linspace(-5, 5, 100)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Sigmoid
axes[0].plot(z, sigmoid(z), 'b-', linewidth=2)
axes[0].set_title('Sigmoid: σ(z) = 1/(1+e^(-z))')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linewidth=0.5)
axes[0].axvline(x=0, color='k', linewidth=0.5)

# ReLU
axes[1].plot(z, relu(z), 'r-', linewidth=2)
axes[1].set_title('ReLU: max(0, z)')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='k', linewidth=0.5)
axes[1].axvline(x=0, color='k', linewidth=0.5)

# Tanh
axes[2].plot(z, tanh(z), 'g-', linewidth=2)
axes[2].set_title('Tanh: (e^z - e^(-z))/(e^z + e^(-z))')
axes[2].grid(True, alpha=0.3)
axes[2].axhline(y=0, color='k', linewidth=0.5)
axes[2].axvline(x=0, color='k', linewidth=0.5)

for ax in axes:
    ax.set_xlabel('z')
    ax.set_ylabel('activation(z)')

plt.tight_layout()
plt.show()

print("✨ These non-linear functions allow neural networks to learn complex patterns!")

## 4. Building a Neural Network from Scratch

### Network Architecture

We'll build a 2-layer neural network:
- **Input layer**: 1 feature (size)
- **Hidden layer**: 4 neurons with ReLU activation
- **Output layer**: 1 neuron (price prediction)

### Forward Propagation

$$
\begin{align}
z^{[1]} &= W^{[1]}x + b^{[1]} \\
a^{[1]} &= \text{ReLU}(z^{[1]}) \\
z^{[2]} &= W^{[2]}a^{[1]} + b^{[2]} \\
\hat{y} &= z^{[2]}
\end{align}
$$

In [ ]:
class SimpleNeuralNetwork:
    """A simple 2-layer neural network for regression"""
    
    def __init__(self, input_size=1, hidden_size=4, output_size=1):
        """Initialize network with random weights"""
        # He initialization for better convergence
        self.W1 = np.random.randn(hidden_size, input_size) * np.sqrt(2/input_size)
        self.b1 = np.zeros((hidden_size, 1))
        self.W2 = np.random.randn(output_size, hidden_size) * np.sqrt(2/hidden_size)
        self.b2 = np.zeros((output_size, 1))
        
        # Store for backpropagation
        self.cache = {}
        
    def forward(self, X):
        """Forward propagation"""
        # Reshape X if needed
        if X.ndim == 1:
            X = X.reshape(1, -1)
        
        # Layer 1
        self.cache['X'] = X.T
        self.cache['z1'] = np.dot(self.W1, X.T) + self.b1
        self.cache['a1'] = relu(self.cache['z1'])
        
        # Layer 2  
        self.cache['z2'] = np.dot(self.W2, self.cache['a1']) + self.b2
        
        return self.cache['z2'].flatten()
    
    def compute_cost(self, y_true, y_pred):
        """Mean Squared Error"""
        m = len(y_true)
        return np.sum((y_pred - y_true)**2) / (2*m)
    
    def backward(self, X, y):
        """Backpropagation - compute gradients"""
        m = len(y)
        
        # Output layer gradients
        y_pred = self.cache['z2'].flatten()
        dz2 = (y_pred - y).reshape(-1, 1) / m
        dW2 = np.dot(dz2, self.cache['a1'].T)
        db2 = np.sum(dz2, axis=1, keepdims=True)
        
        # Hidden layer gradients
        da1 = np.dot(self.W2.T, dz2)
        dz1 = da1 * (self.cache['z1'] > 0)  # ReLU derivative
        dW1 = np.dot(dz1, self.cache['X'].T)
        db1 = np.sum(dz1, axis=1, keepdims=True)
        
        return {'dW1': dW1, 'db1': db1, 'dW2': dW2, 'db2': db2}
    
    def update_parameters(self, gradients, learning_rate):
        """Gradient descent parameter update"""
        self.W1 -= learning_rate * gradients['dW1']
        self.b1 -= learning_rate * gradients['db1']
        self.W2 -= learning_rate * gradients['dW2']
        self.b2 -= learning_rate * gradients['db2']
    
    def train(self, X, y, epochs=1000, learning_rate=0.1, print_every=100):
        """Train the neural network"""
        cost_history = []
        
        for epoch in range(epochs):
            # Forward propagation
            y_pred = self.forward(X)
            
            # Compute cost
            cost = self.compute_cost(y, y_pred)
            cost_history.append(cost)
            
            # Backward propagation
            gradients = self.backward(X, y)
            
            # Update parameters
            self.update_parameters(gradients, learning_rate)
            
            # Print progress
            if epoch % print_every == 0:
                print(f"Epoch {epoch:4d}: Cost = {cost:.4f}")
        
        return cost_history

print("✅ Neural Network class defined!")
print("   - Forward propagation: Input → Hidden → Output")
print("   - Backpropagation: Compute gradients")
print("   - Training: Optimize with gradient descent")

## 5. Train the Neural Network

Now let's train our neural network on the non-linear housing data:

In [ ]:
# Create and train neural network
nn = SimpleNeuralNetwork(input_size=1, hidden_size=8, output_size=1)

print("Training Neural Network...\n")
cost_history = nn.train(x_train, y_train, epochs=2000, learning_rate=0.01, print_every=200)

# Make predictions
y_pred_nn = nn.forward(x_train)
mse_nn = np.mean((y_train - y_pred_nn)**2)

print(f"\n✅ Training complete!")
print(f"Final MSE: {mse_nn:.2f}")
print(f"Improvement over linear: {((mse_linear - mse_nn) / mse_linear * 100):.1f}%")

## 6. Visualize Results

In [ ]:
# Plot comparison
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Left: Model comparison
ax1.scatter(x_train, y_train, alpha=0.5, label='Actual Data', s=30)
ax1.plot(x_train, y_pred_linear, 'r-', linewidth=2, label=f'Linear (MSE={mse_linear:.0f})')
ax1.plot(x_train, y_pred_nn, 'g-', linewidth=2, label=f'Neural Net (MSE={mse_nn:.0f})')
ax1.set_xlabel('Size (100 m²)')
ax1.set_ylabel('Price (1000s CHF)')
ax1.set_title('Model Comparison: Linear vs Neural Network')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Right: Training progress
ax2.plot(cost_history, 'b-', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Cost (MSE)')
ax2.set_title('Training Progress: Cost vs Epochs')
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

plt.tight_layout()
plt.show()

print("\n🎉 The neural network captures the non-linear relationship!")

## 7. Experiment: Deeper Networks

What happens with more layers? More neurons?

Try changing:
- `hidden_size`: Number of neurons (4, 8, 16)
- `learning_rate`: Speed of learning (0.001, 0.01, 0.1)
- `epochs`: Training iterations (1000, 5000, 10000)

In [ ]:
# Experiment with different architectures
hidden_sizes = [2, 4, 8, 16]
results = []

for hidden_size in hidden_sizes:
    nn_exp = SimpleNeuralNetwork(input_size=1, hidden_size=hidden_size, output_size=1)
    cost_hist = nn_exp.train(x_train, y_train, epochs=1000, learning_rate=0.01, print_every=10000)
    y_pred = nn_exp.forward(x_train)
    mse = np.mean((y_train - y_pred)**2)
    results.append({'hidden_size': hidden_size, 'mse': mse, 'predictions': y_pred})
    print(f"Hidden size {hidden_size:2d}: MSE = {mse:.2f}")

# Plot comparison
plt.figure(figsize=(12, 6))
plt.scatter(x_train, y_train, alpha=0.3, s=20, label='Data')

colors = ['r', 'g', 'b', 'm']
for i, result in enumerate(results):
    plt.plot(x_train, result['predictions'], colors[i], 
             label=f"{result['hidden_size']} neurons (MSE={result['mse']:.0f})",
             linewidth=2, alpha=0.7)

plt.xlabel('Size (100 m²)')
plt.ylabel('Price (1000s CHF)')
plt.title('Neural Network Performance vs Architecture')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 8. Key Takeaways

### What We Learned:

1. **Linear models fail on non-linear data** 
   - $f(x) = wx + b$ can only fit straight lines

2. **Activation functions add non-linearity**
   - ReLU, sigmoid, tanh allow learning complex patterns

3. **Neural networks = stacked transformations**
   - Each layer: Linear transformation → Activation
   - Multiple layers → arbitrarily complex functions

4. **Forward propagation computes predictions**
   - Input → Hidden layers → Output

5. **Backpropagation computes gradients**
   - Chain rule to find $\frac{\partial J}{\partial W}$ for each layer
   - Enables gradient descent optimization

6. **More neurons/layers = more capacity**
   - But watch out for overfitting!

---

### The Bridge to Modern AI

This simple 2-layer network with 8 neurons is the **foundation** of:
- **Deep Learning**: 100s of layers, millions of neurons
- **Convolutional Networks**: For image recognition
- **Recurrent Networks**: For sequences (time series, text)
- **Transformers**: The architecture behind GPT, Claude, BERT

**Next:** Notebook 5 will preview how transformers extend these ideas to build language models like ChatGPT!

## 9. Optional Exercises

### Challenge 1: Classification Task
Modify the network to classify housing prices as "cheap" (<300k) or "expensive" (>300k).
Hint: Use sigmoid activation in output layer!

### Challenge 2: Different Activation Functions
Replace ReLU with sigmoid or tanh in hidden layer. How does performance change?

### Challenge 3: Learning Rate Impact
Try learning rates of 0.001, 0.01, 0.1, 1.0. What happens?

### Challenge 4: Add a Third Layer
Extend the network to have 2 hidden layers. Does it improve?

---

## References & Further Reading

- **Neural Networks and Deep Learning** by Michael Nielsen
- **Deep Learning** by Goodfellow, Bengio, Courville
- **3Blue1Brown Neural Network Series** (YouTube)
- **Stanford CS231n** Convolutional Neural Networks